# Alice CID categorization sheet export

The *gestão de saúde magenta* squad inputs dw weights for daly calculations manually in a google sheet. This notebook is to export the sheet, after downloading from the Google Drive and storing it in a local computer. Since the sheet is updated not very often, there is not much gain from automating the process in Airflow (*yet*).

Steps:

    - Download the google sheet DALY weights and store it in the 'redshift_import_export\external\' package folder;

    - run this notebook to upload to Redshift;
    
    - use the DW to make calculations, new tables, dashes...
***

## 0. Setup

In [5]:
#connection to redshift
from redshift_import_export.interfaces.data_interactor import DataInteractor
di = DataInteractor()
import requests


#standard packages
import warnings
warnings.filterwarnings('ignore')
import os
import collections
import pandas as pd
import numpy as np



#time packages
import datetime as dt

## 1. Import

In [6]:
df1 = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/input_alice_cids_one.csv')
df2 = pd.read_csv(os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/external/input_alice_cids_two.csv')

In [7]:
df1.columns

Index(['CID 10', 'agudo ou longitudinal?'], dtype='object')

In [8]:
df2.columns

Index(['Unnamed: 0', 'Disease.Code.Value', 'capitulo', 'grupos', 'categorias',
       'subcategorias', 'VOLUME', 'Categoria - Sugestão Bia',
       'Categoria - Sugestão Janos', 'Categoria - ALICE', 'Categoria 2.0',
       'Protocolo AA?', 'Obs'],
      dtype='object')

## 2. Transform

In [10]:
df1.rename(columns = {'CID 10': 'cid', 'agudo ou longitudinal?': 'alice_acute_class'}, inplace = True)

In [11]:
df2.rename(columns={'Disease.Code.Value': 'cid', 'Categoria 2.0': 'alice_category', 'capitulo': 'chapter', 'grupos': 'group', 'categorias': 'category', 'subcategorias': 'subcategory'}, inplace = True)
df2 = df2[['cid', 'alice_category', 'chapter', 'group', 'category', 'subcategory']]


In [12]:
df3 = df2.merge(df1, how = 'left', on = 'cid')
df3.drop_duplicates(inplace = True)
df3.head(10)

,cid,alice_category,chapter,group,category,subcategory,alice_acute_class
0,K35,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Doenças do apêndice,Apendicite aguda,Apendicite aguda,NaN
1,K37,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Doenças do apêndice,"Apendicite, sem outras especificações","Apendicite, sem outras especificações",agudo
2,K36,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Doenças do apêndice,Outras formas de apendicite,Outras formas de apendicite,agudo
3,K561,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,Intussuscepção,longitudinal
4,K562,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,Volvo,longitudinal
5,K564,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,Outras obstruções do intestino,agudo
6,K565,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,Aderências intestinais (bridas) com obstrução,longitudinal
7,K566,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,"Outras formas de obstrução intestinal, e as nã...",longitudinal
8,K56,Abdome agudo,Capítulo XI - Doenças do aparelho digestivo,Outras doenças dos intestinos,Íleo paralítico e obstrução intestinal sem hérnia,Íleo paralítico e obstrução intestinal sem hérnia,NaN
9,F101,Abuso crônico de substâncias,Capítulo V - Transtornos mentais e comportamen...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,Transtornos mentais e comportamentais devidos ...,longitudinal


## 3. Export

In [13]:
di.redshift.insert_table(df3, table_name='alice_cid_categories', if_exists='replace')

Table alice_cid_categories inserted successfully, with 2531 rows.
